# Librerías

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException
import time
import re
import pandas as pd

# Inicio de sesión

In [ ]:
# Cargar el driver de Firefox
driver = webdriver.Firefox()
# Establecer la pagina a ingresar
driver.get("http://www.facebook.com")
# Seleccionar username y password
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))
# Limpiar username y password
username.clear()
password.clear()
# Enviar correo y contraseña
username.send_keys("su_correo")
password.send_keys("su_contrasena")
# Iniciar sesion
log_in = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

# Función para extraer amigos

In [ ]:
def get_friendship(path_url, i, n):
    status_person = ["friends", "following", "followers"]
    driver.get(path_url + "/" + status_person[0])

    for j in range(0,n):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(10)

    link_user = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".sbcfpzgs > div:nth-child(1)")))
    link_user = link_user.find_elements_by_tag_name('a')
    link_user = [a.get_attribute('href') for a in link_user]
    link_user = list(dict.fromkeys(link_user))
    link_user = [a for a in link_user if a != None]
    link_user = [a for a in link_user if a != path_url + "/" + status_person[0]]
    link_user = [a for a in link_user if a != "https://www.facebook.com/friends/"]
    link_user = [a for a in link_user if a.find("mutual") == -1]
    
    return(link_user)

# Función para extraer información de usuario

In [ ]:
def get_profile_desc(path_url):
    # url
    driver.get(path_url)
    time.sleep(5)
    
    # username
    try:
        name_user = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".l1jc4y16 > h1:nth-child(1)"))).text
    except:
        name_user = ''
    # numero de amigos
    try:
        friends = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.ll8tlv6m:nth-child(2) > span:nth-child(1) > a:nth-child(1)"))).text
    except:
        friends = ''
    # descripcion de perfil
    try:
        description = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.lpgh02oy:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)"))).text
    except:
        description = ''
        
    profile = {'user name': name_user
               ,'friends': friends
               ,'description': description
              }
    
    return(profile)

# Extracción de amigos

In [ ]:
# Establecer el usuario de interes
path_url = "https://www.facebook.com/jairinrb"

In [ ]:
# Informacion de usuario
info = []
info.append(get_profile_desc(path_url))

In [ ]:
# Informacion de mi perfil, yo lo tengo privado
info[0]

In [ ]:
# Extracción de amigos
# 0 se corresponde con friends
# 10 con el número de veces que se desea desplazar la página
amigos = get_friendship(path_url, 0, 10)

In [ ]:
# Cantidad de amigos extraídos, al momento de obtención tenía 452 amigos
len(amigos)

In [ ]:
# Uniendo mi información con la de mis amigos
info = info + [get_profile_desc(a) for a in amigos]

In [ ]:
# Cantidad total de usuarios cuya información se ha obtenido
len(info)

In [ ]:
# Creación de un dataframe con la información capturada
df = pd.DataFrame(info)

In [ ]:
# Se crea la columna link, que será el indentificador de cada usuario
df['link'] = [path_url] + amigos

In [ ]:
# una muestra de los primeros resultados
df.head(5)

In [ ]:
# Las columans obtenidas
df.columns

In [ ]:
# Renombrando columanas
df.columns = ['Name','Friend','Description','link']

In [ ]:
df.head(5)

In [ ]:
# Exportación de resultados a un archivo csv
df.to_csv('amigos.csv',index=False, encoding='utf-8', sep = ';')

Elaborado por Jairo Rojas

Email: jairinrojasb@gmail.com